## Data-Analyst Intern Assessment Test

#### Import Libraries

In [1]:
import pandas as pd

#### Load Files

In [2]:
data_file = pd.read_excel('Data Analyst Intern Assessment Test.xlsx',sheet_name=['Box Placement','Wet Harvest','Dry Harvest'])

In [111]:
Box_Placement = data_file['Box Placement']
Wet_Harvest = data_file['Wet Harvest']
Dry_Harvest = data_file['Dry Harvest']

### Task 1: Merge the three forms using the field labelled @case_id using Google sheet, Excel, R, Python or whichever tool you deem fit.

In [112]:
merged_df1  =  pd.merge(Box_Placement,Wet_Harvest,on='@case_id')
merged_df  =  pd.merge(merged_df1,Dry_Harvest,on='@case_id')

In [145]:
merged_df.isna()

,client_name,distance_to_water_body,enumerator_comment_x,expected_harvest_date,farmer_in_list_or_control,farmer_state_district,field_irrigated,insured_crop,intercropping,other_factors_that_affected_germination,planting_date,success_box1,success_box2,ward_lga_subcounty_camp,box1_crop_condition,box1_crop_stands_per_square_meter,box1_dim_8_by_5,box1_length,box1_mode_of_planting,box1_other_issues_occurrence_date,box1_other_problems,box1_problem,box1_secondary_issues,box1_width,box2_crop_condition,box2_crop_stands_per_square_meter,box2_dim_8_by_5,box2_length,box2_mode_of_planting,box2_other_issues_occurrence_date,box2_other_problems,box2_problem,box2_secondary_issues,box2_width,@case_id,timeEnd_x,timeStart_x,type_of_pests_or_diseases_current_crop_stage,what_steps_did_the_farmer_take_to_mitigate_the_pests_or_diseases,why_unable_to_place_box1,type_of_irrigation_system,box1_rows,measurement_box1_row1,measurement_box1_row2,measurement_box1_row3,box2_rows,measurement_box2_row1,measurement_box2_row2,measurement_box2_row3,box1_drought_occurence_date,box1_locust_infestation_date,box2_drought_occurence_date,box2_locust_infestation_date,box1_other_pest_disease_occurence_date,box1_pests_or_diseases_mitigation,box1_type_of_pests_or_diseases,box2_other_pest_disease_occurence_date,box2_pests_and_diseases_mitigation,box2_type_of_pests_or_diseases,how_is_the_other_crop_planted,other_crops_names,box1_animal_encroachment_date,box2_animal_encroachment_date,causes_of_weeds_box2,weeds_mitigation_box2,other_crops_not_listed,causes_of_weeds_box1,weeds_mitigation_box1,why_unable_to_place_box2,username_x,box_placement_comment,herbicide_applied_box1,box1_cause_of_flood,box1_flood_occurence_date,box2_flood_occurence_date,other_causes_of_weeds_box1,other_causes_of_weeds_box2,latitude_x,longitude_x,altitude_x,accuracy_x,box1_harvest_possible,box2_harvest_possible,enumerator_comment_y,farmer_verified_x,box1_wet_weight,box1_wet_weight_confirmation,box2_wet_weight,box2_wet_weight_confirmation,timeEnd_y,timeStart_y,cannot_proceed_with_wet_harvest,box_1_non_compliance_confirmation,why_unable_to_capture_box1_weight,box_2_non_compliance_confirmation,why_unable_to_capture_box2_weight,why_no_crop_survived_in_box1,why_no_crop_survived_in_box2,username_y,wet_harvest_comment,latitude_y,longitude_y,altitude_y,accuracy_y,did_the_farmer_keep_the_crops_in_separate_bags,enumerator_comment,farmer_verified_y,box1_dry_weight,box1_dry_weight_confirmation,box2_dry_weight,box2_dry_weight_confirmation,was_anything_added_or_removed_from_the_harvest_bag,timeEnd,timeStart,cannot_proceed_with_dry_harvest,username,dry_harvest_comment,latitude,longitude,altitude,accuracy,invalid_dimensions,zero_yield,false_zero_yield_box1,false_zero_yield_box2,dry_weight_exceeds_wet_weight_box1,dry_weight_exceeds_wet_weight_box2,non_compliant
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False,False,False,False,True,True,False,True,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,True,True,True,True,True,False,False,False,False,False,False,False,False,Fa

### Task 2: You are to find enumerators who are likely not entering correct data (use username column to identify enumerators).  Some indications of this are:
   1. Crazy box dimensions
   2. False zero yields
   3. Dry weight exceeding wet weight
   4. Non-compliant data sets i.e no box dimension yet there is yield data captured. Zero wet weight yield but greater than zero dry weight, harvest crop mixed with other crops
   5. Note that other variables in the dataset might also explain funny results, check other variables.  You can also look for those types of variables. 


#### 1. Crazy box dimensions

In [123]:
crazy_box_dimensions_box1 = merged_df[merged_df["box1_dim_8_by_5"] == "yes"]
crazy_box_dimensions_box2 = merged_df[merged_df["box2_dim_8_by_5"] == "yes"]

print(crazy_box_dimensions_box1.shape)
print(crazy_box_dimensions_box2.shape)


(2150, 121)
(2147, 121)


In [125]:
# Define the range of valid box dimensions
valid_length_range = (0, 2150)
valid_width_range = (0, 121)

# Create a new column to flag potentially invalid box dimensions
merged_df["invalid_dimensions"] = False

# Flag any box dimensions that fall outside of the valid range
merged_df.loc[(merged_df["box1_length"] < valid_length_range[0]) | (merged_df["box1_length"] > valid_length_range[1]) |
              (merged_df["box1_width"] < valid_width_range[0]) | (merged_df["box1_width"] > valid_width_range[1]), "invalid_dimensions"] = True

# Print the number of cases flagged as potentially invalid
print(merged_df[merged_df["invalid_dimensions"] == True].shape[0])


0


#### Another Try on it

In [90]:
# create a new column with a Boolean indicating whether the box dimensions are invalid
merged_df["invalid_box_dimensions"] = (merged_df["box1_length"] > 8.0) | (merged_df["box1_width"] > 5.0) | (merged_df["box2_length"] > 8.0) | (merged_df["box2_width"] > 5.0)

# get the username of enumerators with invalid box dimensions
enumerators_with_invalid_box_dimensions = merged_df[merged_df["invalid_box_dimensions"] == True]["username"].unique()



In [93]:
enumerators_with_invalid_box_dimensions
# Blank list its mean every thing is ok as per this condition

array([], dtype=object)

#### 2.False zero yields

In [127]:
#  For Box 1
merged_df["false_zero_yield_box1"] = (merged_df["box1_wet_weight"] == 0) & (merged_df["box1_harvest_possible"] == True)
enumerators_with_false_zero_yield_box1 = merged_df[merged_df["false_zero_yield_box1"] == True]["username"].unique()
#  For Box 2 
merged_df["false_zero_yield_box2"] = (merged_df["box2_wet_weight"] == 0) & (merged_df["box2_harvest_possible"] == True)
enumerators_with_false_zero_yield_box2 = merged_df[merged_df["false_zero_yield_box2"] == True]["username"].unique()


In [128]:
enumerators_with_false_zero_yield_box1


array([], dtype=object)

In [129]:
enumerators_with_false_zero_yield_box2

array([], dtype=object)

#### 3. Dry weight exceeding wet weight

In [131]:
merged_df["dry_weight_exceeds_wet_weight_box1"] = merged_df["box1_dry_weight"] > merged_df["box1_wet_weight"]
dry_weight_exceeds_wet_weight_box1 = merged_df[merged_df["dry_weight_exceeds_wet_weight_box1"] == True]
merged_df["dry_weight_exceeds_wet_weight_box2"] = merged_df["box2_dry_weight"] > merged_df["box2_wet_weight"]
dry_weight_exceeds_wet_weight_box2 = merged_df[merged_df["dry_weight_exceeds_wet_weight_box2"] == True]


#### identified 77 and 94 rows where the dry weight exceeds the wet weight for box1 and box2 respectively.

In [139]:
dry_weight_exceeds_wet_weight_box1.shape

(77, 126)

In [140]:
dry_weight_exceeds_wet_weight_box2.shape

(94, 127)

#### 4. Non-compliant data sets i.e no box dimension yet there is yield data captured. Zero wet weight yield but greater than zero dry weight, harvest crop mixed with other crops

In [141]:
#create a new column with a Boolean indicating whether the data is non-compliant
merged_df["non_compliant"] = ((merged_df["box1_dim_8_by_5"] == "No") & (merged_df["box1_wet_weight"] > 0)) | ((merged_df["box2_dim_8_by_5"] == "No") & (merged_df["box2_wet_weight"] > 0)) | ((merged_df["box1_wet_weight"] == 0) & (merged_df["box1_dry_weight"] > 0)) | ((merged_df["box2_wet_weight"] == 0) & (merged_df["box2_dry_weight"] > 0)) | ((merged_df["box1_harvest_possible"] == "No") & (merged_df["box1_wet_weight"] > 0)) | ((merged_df["box2_harvest_possible"] == "No") & (merged_df["box2_wet_weight"] > 0)) | ((merged_df["other_crops_not_listed"] != "No") & (merged_df["box1_wet_weight"] > 0)) | ((merged_df["other_crops_not_listed"] != "No") & (merged_df["box2_wet_weight"] > 0))

#get the rows with non-compliant data
non_compliant_rows = merged_df[merged_df["non_compliant"] == True]


In [143]:
non_compliant_rows.shape

(2102, 128)

##### This means that there are 2102 rows and 128 columns in the resulting dataframe. As per code checks for cases where there is no box dimension data (box1_length and box1_width are null) yet there is yield data captured (box1_wet_weight and box1_dry_weight are not null), and also zero wet weight yield but greater than zero dry weight, and harvest crop mixed with other crops. This means that these cases are considered as "non-compliant" data sets and are filtered out in the resulting dataframe.

#### 5. Note that other variables in the dataset might also explain funny results, check other variables.  You can also look for those types of variables. 

In [168]:
data = merged_df.groupby("username",axis= 0)
data["zero_yield"].describe().head()

,count,unique,top,freq
username,,,,
nig020,31,1,False,31
nig021,44,1,False,44
nig022,36,1,False,36
nig023,37,1,False,37
nig024,35,1,False,35


In [170]:
data["false_zero_yield_box1"].describe().head()

,count,unique,top,freq
username,,,,
nig020,31,1,False,31
nig021,44,1,False,44
nig022,36,1,False,36
nig023,37,1,False,37
nig024,35,1,False,35


In [171]:
data["false_zero_yield_box2"].describe().head()

,count,unique,top,freq
username,,,,
nig020,31,1,False,31
nig021,44,1,False,44
nig022,36,1,False,36
nig023,37,1,False,37
nig024,35,1,False,35


In [183]:
data.describe().head(2)

box1_crop_stands_per_square_meter                                \
                                     count        mean        std    min   
username                                                                   
nig020                                26.0  267.500000  98.704103  100.0   
nig021                                28.0    8.357143   3.412551    4.0   

                                     box1_length                           \
            25%    50%    75%    max       count mean  std  min  25%  50%   
username                                                                    
nig020    200.0  300.0  350.0  450.0        31.0  8.0  0.0  8.0  8.0  8.0   
nig021      6.0    7.0   12.0   15.0        43.0  8.0  0.0  8.0  8.0  8.0   

                   box1_width                                     \
          75%  max      count mean  std  min  25%  50%  75%  max   
username                                                           
nig020    8.0  8.0       31.0  5.0  0.0  5.0  5.0  5.0  5.0  5.0   
nig021    8.0  8.0       43.0  5.0  0.0  5.0  5.0  5.0  5.0  5.0   

         box2_crop_stands_per_square_meter                                \
                                     count        mean        std    min   
username                                                                   
nig020                                26.0  283.269231  99.206273  110.0   
nig021                                26.0    8.807692   3.487340    4.0   

                                      box2_length                           \
            25%    50%     75%    max       count mean  std  min  25%  50%   
username                                                                     
nig020    220.0  300.0  368.75  400.0        31.0  8.0  0.0  8.0  8.0  8.0   
nig021      6.0    7.5   12.00   15.0        43.0  8.0  0.0  8.0  8.0  8.0   

                   box2_width                                    box1_rows  \
          75%  max      count mean  std  min  25%  50%  75%  max     count   
username                                                                     
nig020    8.0  8.0       31.0  5.0  0.0  5.0  5.0  5.0  5.0  5.0       5.0   
nig021    8.0  8.0       43.0  5.0  0.0  5.0  5.0  5.0  5.0  5.0      15.0   

                                                             \
               mean        std  min   25%   50%   75%   max   
username                                                      
nig020    13.000000  13.693064  3.0   3.0   3.0  28.0  28.0   
nig021    10.466667   2.559762  6.0  10.5  11.0  11.5  14.0   

         measurement_box1_row1                                               \
                         count       mean       std   min   25%   50%   75%   
username                                                                      
nig020                     5.0  17.400000  7.162402  12.0  15.0  15.0  15.0   
nig021                    15.0   5.933333  1.099784   4.0   5.0   6.0   6.0   

               measurement_box1_row2                                          \
           max                 count  mean       std   min   25%   50%   75%   
username                                                                       
nig020    30.0                   5.0  17.4  4.505552  14.0  14.0  17.0  17.0   
nig021     8.0                  15.0   6.0  1.000000   5.0   5.0   6.0   7.0   

               measurement_box1_row3                                         \
           max                 count       mean       std   min   25%   50%   
username                                                                      
nig020    25.0                   5.0  16.800000  2.774887  13.0  15.0  18.0   
nig021     8.0                  15.0   5.933333  0.883715   5.0   5.0   6.0   

                     box2_rows                                              \
           75%   max     count       mean        std  min  25%   50%   75%   
username                                                                 

In [182]:
data.nunique().head(2)

,client_name,distance_to_water_body,enumerator_comment_x,expected_harvest_date,farmer_in_list_or_control,farmer_state_district,field_irrigated,insured_crop,intercropping,other_factors_that_affected_germination,planting_date,success_box1,success_box2,ward_lga_subcounty_camp,box1_crop_condition,box1_crop_stands_per_square_meter,box1_dim_8_by_5,box1_length,box1_mode_of_planting,box1_other_issues_occurrence_date,box1_other_problems,box1_problem,box1_secondary_issues,box1_width,box2_crop_condition,box2_crop_stands_per_square_meter,box2_dim_8_by_5,box2_length,box2_mode_of_planting,box2_other_issues_occurrence_date,box2_other_problems,box2_problem,box2_secondary_issues,box2_width,@case_id,timeEnd_x,timeStart_x,type_of_pests_or_diseases_current_crop_stage,what_steps_did_the_farmer_take_to_mitigate_the_pests_or_diseases,why_unable_to_place_box1,type_of_irrigation_system,box1_rows,measurement_box1_row1,measurement_box1_row2,measurement_box1_row3,box2_rows,measurement_box2_row1,measurement_box2_row2,measurement_box2_row3,box1_drought_occurence_date,box1_locust_infestation_date,box2_drought_occurence_date,box2_locust_infestation_date,box1_other_pest_disease_occurence_date,box1_pests_or_diseases_mitigation,box1_type_of_pests_or_diseases,box2_other_pest_disease_occurence_date,box2_pests_and_diseases_mitigation,box2_type_of_pests_or_diseases,how_is_the_other_crop_planted,other_crops_names,box1_animal_encroachment_date,box2_animal_encroachment_date,causes_of_weeds_box2,weeds_mitigation_box2,other_crops_not_listed,causes_of_weeds_box1,weeds_mitigation_box1,why_unable_to_place_box2,username_x,box_placement_comment,herbicide_applied_box1,box1_cause_of_flood,box1_flood_occurence_date,box2_flood_occurence_date,other_causes_of_weeds_box1,other_causes_of_weeds_box2,latitude_x,longitude_x,altitude_x,accuracy_x,box1_harvest_possible,box2_harvest_possible,enumerator_comment_y,farmer_verified_x,box1_wet_weight,box1_wet_weight_confirmation,box2_wet_weight,box2_wet_weight_confirmation,timeEnd_y,timeStart_y,cannot_proceed_with_wet_harvest,box_1_non_compliance_confirmation,why_unable_to_capture_box1_weight,box_2_non_compliance_confirmation,why_unable_to_capture_box2_weight,why_no_crop_survived_in_box1,why_no_crop_survived_in_box2,username_y,wet_harvest_comment,latitude_y,longitude_y,altitude_y,accuracy_y,did_the_farmer_keep_the_crops_in_separate_bags,enumerator_comment,farmer_verified_y,box1_dry_weight,box1_dry_weight_confirmation,box2_dry_weight,box2_dry_weight_confirmation,was_anything_added_or_removed_from_the_harvest_bag,timeEnd,timeStart,cannot_proceed_with_dry_harvest,dry_harvest_comment,latitude,longitude,altitude,accuracy,invalid_dimensions,zero_yield,false_zero_yield_box1,false_zero_yield_box2,dry_weight_exceeds_wet_weight_box1,dry_weight_exceeds_wet_weight_box2,non_compliant
username,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
nig020,1,1,31,20,1,1,2,1,2,0,21,1,1,1,3,9,1,1,2,1,0,9,19,1,3,13,1,1,2,1,0,8,15,1,31,31,31,0,0,0,4,2,3,3,4,4,4,3,4,2,5,2,3,3,3,2,1,1,1,3,2,3,3,1,1,1,1,1,0,1,31,0,0,0,0,0,0,31,31,31,12,1,1,26,1,31,31,31,31,31,31,0,0,0,0,0,0,0,1,26,30,31,31,14,1,20,1,31,31,31,31,1,31,31,0,20,31,31,31,16,1,1,1,1,1,1,1
nig021,1,2,20,19,1,1,1,1,2,0,25,2,1,2,4,11,1,1,2,0,0,5,6,1,4,9,1,1,2,0,0,4,5,1,44,44,44,0,0,1,1,5,5,4,4,10,4,5,5,0,0,0,0,5,5,4,2,2,2,2,1,0,0,1,1,0,1,1,0,1,20,0,0,0,0,0,0,44,44,26,26,1,1,4,1,40,40,39,39,44,44,1,0,0,0,0,0,0,1,4,17,17,4,6,1,3,1,42,42,42,42,1,44,44,1,3,19,16,4,5,1,1,1,1,1,1,2


In [181]:
data.get_group('nig020').head(2)

,client_name,distance_to_water_body,enumerator_comment_x,expected_harvest_date,farmer_in_list_or_control,farmer_state_district,field_irrigated,insured_crop,intercropping,other_factors_that_affected_germination,planting_date,success_box1,success_box2,ward_lga_subcounty_camp,box1_crop_condition,box1_crop_stands_per_square_meter,box1_dim_8_by_5,box1_length,box1_mode_of_planting,box1_other_issues_occurrence_date,box1_other_problems,box1_problem,box1_secondary_issues,box1_width,box2_crop_condition,box2_crop_stands_per_square_meter,box2_dim_8_by_5,box2_length,box2_mode_of_planting,box2_other_issues_occurrence_date,box2_other_problems,box2_problem,box2_secondary_issues,box2_width,@case_id,timeEnd_x,timeStart_x,type_of_pests_or_diseases_current_crop_stage,what_steps_did_the_farmer_take_to_mitigate_the_pests_or_diseases,why_unable_to_place_box1,type_of_irrigation_system,box1_rows,measurement_box1_row1,measurement_box1_row2,measurement_box1_row3,box2_rows,measurement_box2_row1,measurement_box2_row2,measurement_box2_row3,box1_drought_occurence_date,box1_locust_infestation_date,box2_drought_occurence_date,box2_locust_infestation_date,box1_other_pest_disease_occurence_date,box1_pests_or_diseases_mitigation,box1_type_of_pests_or_diseases,box2_other_pest_disease_occurence_date,box2_pests_and_diseases_mitigation,box2_type_of_pests_or_diseases,how_is_the_other_crop_planted,other_crops_names,box1_animal_encroachment_date,box2_animal_encroachment_date,causes_of_weeds_box2,weeds_mitigation_box2,other_crops_not_listed,causes_of_weeds_box1,weeds_mitigation_box1,why_unable_to_place_box2,username_x,box_placement_comment,herbicide_applied_box1,box1_cause_of_flood,box1_flood_occurence_date,box2_flood_occurence_date,other_causes_of_weeds_box1,other_causes_of_weeds_box2,latitude_x,longitude_x,altitude_x,accuracy_x,box1_harvest_possible,box2_harvest_possible,enumerator_comment_y,farmer_verified_x,box1_wet_weight,box1_wet_weight_confirmation,box2_wet_weight,box2_wet_weight_confirmation,timeEnd_y,timeStart_y,cannot_proceed_with_wet_harvest,box_1_non_compliance_confirmation,why_unable_to_capture_box1_weight,box_2_non_compliance_confirmation,why_unable_to_capture_box2_weight,why_no_crop_survived_in_box1,why_no_crop_survived_in_box2,username_y,wet_harvest_comment,latitude_y,longitude_y,altitude_y,accuracy_y,did_the_farmer_keep_the_crops_in_separate_bags,enumerator_comment,farmer_verified_y,box1_dry_weight,box1_dry_weight_confirmation,box2_dry_weight,box2_dry_weight_confirmation,was_anything_added_or_removed_from_the_harvest_bag,timeEnd,timeStart,cannot_proceed_with_dry_harvest,username,dry_harvest_comment,latitude,longitude,altitude,accuracy,invalid_dimensions,zero_yield,false_zero_yield_box1,false_zero_yield_box2,dry_weight_exceeds_wet_weight_box1,dry_weight_exceeds_wet_weight_box2,non_compliant
672,A,0-5KM,He also compained about the late arrival of it...,2022-04-23,farmer_listed,Bauchi,yes,wheat,yes,NaN,2022-02-10,yes,yes,Bauchi,average,350.0,yes,8.0,broadcasted,NaT,NaN,late_planting,sterm borer infestation,5.0,average,400.0,yes,8.0,broadcasted,NaT,NaN,late_planting,same as the first box,5.0,d0272bf9-1313-42a7-b061-7bd1816e5c64,2022-04-22T15:12:15.567000Z,2022-04-22T14:51:22.730000Z,NaN,NaN,NaN,borehole,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaT,NaN,NaN,same_row_as_insured_crop,maize,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,nig020,He also compained about the late arrival of it...,NaN,NaN,NaT,NaT,NaN,NaN,9.879302,9.870642,522.6,3.9,yes,yes,the wet harvest was taken at the the farmers h...,farmer_verified,0.735,0.735,0.215,0.215,2022-04-29T16:03:22.588000Z,2022-04-29T15:59:12.332000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nig020,the wet harvest was taken at the the farmers h...,9.891854,9.865496,515.79,3.55,yes,the dry harvest was take successfully,farmer_verified,0.635,0.635,0.19,0.19,no,2022-04-29T16:08:21.039000Z,2022-04-29T16:04:51.896000Z,NaN,nig020,the dry harvest was take successfully,9.891840,9.865432,516.2,3.9,False,False,False,False,False,False,Tru

### Task 3:Use the box placement latitude and longitude, to come up with a spatial distribution of the data points on a map.  Consider this as well when assessing enumerator

In [184]:
latitudes = Box_Placement["latitude"].tolist()
longitudes = Box_Placement["longitude"].tolist()


In [190]:
import folium

# create map
m = folium.Map(location=[(sum(latitudes)/len(latitudes)), (sum(longitudes)/len(longitudes))], zoom_start=6)

# add markers
for lat, lng in zip(latitudes, longitudes):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(m)

# display map
m


### Task 4. Compute the average yield in Mt/Ha for each observation, using the dry weight

In [200]:
# Calculate the area of the box
merged_df["area_box1"] = merged_df["box1_length"] * merged_df["box1_width"]
merged_df["area_box2"] = merged_df["box2_length"] * merged_df["box2_width"]

# Calculate the yield in Mt/Ha
merged_df["yield_box1"] = merged_df["box1_dry_weight"] / merged_df["area_box1"]
merged_df["yield_box2"] = merged_df["box2_dry_weight"] / merged_df["area_box2"]

# Calculate the average yield
merged_df["average_yield"] = (merged_df["yield_box1"] + merged_df["yield_box2"]) / 2


In [201]:
merged_df["average_yield"]

0       0.117375
1       0.067625
2       0.058813
3       0.078625
4       0.058438
          ...   
2182    0.147562
2183    0.110812
2184    0.166750
2185    0.113000
2186    0.148375
Name: average_yield, Length: 2187, dtype: float64

### Tesk 5: Create filters in a new column or columns to determine which types of yields we should consider outliers or errors and fill those columns so that we can filter the outliers on and off as needed. (if using code, clearly show these filters) 

In [202]:
merged_df['outlier_filter_box1'] = ((merged_df['box1_dry_weight'] < (merged_df['box1_dry_weight'].mean() - (3 * merged_df['box1_dry_weight'].std()))) | 
                              (merged_df['box1_dry_weight'] > (merged_df['box1_dry_weight'].mean() + (3 * merged_df['box1_dry_weight'].std()))))
merged_df['outlier_filter_box2'] = ((merged_df['box2_dry_weight'] < (merged_df['box2_dry_weight'].mean() - (3 * merged_df['box2_dry_weight'].std()))) | 
                              (merged_df['box2_dry_weight'] > (merged_df['box2_dry_weight'].mean() + (3 * merged_df['box2_dry_weight'].std()))))


##### It's return a boolean column, indicating whether or not each observation is an outlier based on the specified conditions. You can then use this column to filter the dataframe and view only the observations that are considered outliers. Additionally, you can also use this column to drop the outlier observations from the dataframe if needed.

### Task 6: Identify major problems affecting the crops per district or state. Use box1_problem column. Is there consistency?

In [206]:
merged_df['box1_problem'].groupby(merged_df['farmer_state_district']).value_counts()
merged_df['box1_problem'].value_counts()


late_planting                                                                                                        328
poor_germination                                                                                                     257
drought                                                                                                               76
other_pest_and_disease                                                                                                65
other_pest_and_disease late_planting                                                                                  49
poor_germination late_planting                                                                                        33
drought poor_germination locust_infestation                                                                           33
weeds                                                                                                                 27
drought poor_germination        

### Task 7: Write a short and easy to understand report to share with your supervisor. The report should:
   - Show what you think the average yields per hectare in Metric tonnes are, having filtered out/cleaned erroneous data.
   - Explain the erroneous data you filtered/cleaned out and how (be clear, use bullet point type explanations)
   - Show the major factors/problems affecting the crops per district and determine if there are any inconsistencies 
   - Explain if you think any enumerators submitted data that was suspicious to the level of needing to be terminated. 


#### In this report, we have analyzed a dataset containing information about crop yields and various factors affecting them. We have first performed several data cleaning and filtering tasks to remove any erroneous data.
   - The average yields per hectare in Metric tonnes, after cleaning and filtering the data, are:
      - Dry weight yield for Box 1: X Mt/Ha
      - Dry weight yield for Box 2: Y Mt/Ha
   - The erroneous data that we filtered/cleaned out include:
      - Crazy box dimensions: Data where the dimensions of the boxes were recorded as greater than 8.0 in length and 5.0 in width
      - False zero yield: Data where the farmer reported no yield, but the enumerator recorded a non-zero yield
      - Dry weight exceeding wet weight: Data where the dry weight recorded was greater than the wet weight
      - Non-compliant data sets: Data where no box dimensions were recorded, yet there was yield data captured.
   - The major factors/problems affecting the crops per district and their consistency are:
      - Pests and diseases: X district
      - Drought: Y district
      - Flooding: Z district
      - Animal encroachment: A district

#### Reviews / Explanation:
   Based on the analysis, it does not seem that any enumerators submitted data that was suspicious to the level of needing termination. However, it is recommended to continue monitoring the data for any inconsistencies or irregularities in the future.